In [1]:
!pip install -q torch torchvision torchaudio
!pip install -q pandas scikit-learn nltk transformers sacrebleu datasets pyarabic

print("Bibliothèques installées (PyArabic pour normalisation arabe)")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 835.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 6.9 MB/s eta 0:00:00
Bibliothèques installées (PyArabic pour normalisation arabe)


In [2]:
from datasets import load_dataset
import pandas as pd
import random

dataset = load_dataset("khalidalt/SANAD", split="train")

tech_articles = [ex for ex in dataset if ex['category'] == 'Tech']
print(f"{len(tech_articles)} articles Tech chargés")

texts = [ex['article'] for ex in tech_articles]
scores = [round(random.uniform(6.0, 10.0), 1) for _ in range(len(texts))]

df = pd.DataFrame({"Text": texts, "Score": scores})
df = df.sample(n=4000, random_state=42).reset_index(drop=True)

df.to_csv("arabic_dataset.csv", index=False, encoding='utf-8-sig')
print(f"Dataset créé : {len(df)} articles")
df.head()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.json:   0%|          | 0.00/336M [00:00<?, ?B/s]

test.json:   0%|          | 0.00/37.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/99810 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11090 [00:00<?, ? examples/s]

15210 articles Tech chargés
Dataset créé : 4000 articles


,Text,Score
0,\nCat S60 .. أول هاتف ذكي في العالم مع كاميرا ...,7.3
1,"\n""فيس بوك"" تتيح لمستخدميها التقاط صور بزاوية ...",7.0
2,\nدبي حمدي سعد: نما عدد اشتراكات المستهلكين ال...,8.1
3,\nتشارك شركة بوليكوم، في معرض جيتكس في دورته ا...,8.6
4,\n«إل جي» تمنح مستخدميها هاتفها «G4» الذكي مجا...,9.8


In [3]:
import re
import pyarabic.araby as araby
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('arabic'))

df = pd.read_csv("arabic_dataset.csv")

def preprocess(text):
    if not isinstance(text, str): return ""
    # Suppression diacritiques + normalisation (alef, teh, etc.)
    text = araby.strip_diacritics(text)
    text = araby.normalize_alef(text)
    text = araby.normalize_teh(text)
    text = araby.normalize_hamza(text)

    # Nettoyage basique (ponctuation, chiffres optionnel)
    text = re.sub(r'[^\w\s]', ' ', text)  # Remplace ponctuation par espace
    text = re.sub(r'\d+', '', text)       # Supprime chiffres

    # Tokenization simple + stop words
    tokens = araby.tokenize(text)
    tokens = [t for t in tokens if t not in stop_words and len(t) > 2]
    return ' '.join(tokens)

print("Préprocessing simple en cours...")
df['Processed_Text'] = df['Text'].apply(preprocess)
df = df[df['Processed_Text'].str.len() > 30].reset_index(drop=True)
df.to_csv("preprocessed_dataset.csv", index=False, encoding='utf-8-sig')
print(f"Prétraité : {len(df)} textes conservés")
df[['Text', 'Processed_Text']].head(3)

Préprocessing simple en cours...
Prétraité : 4000 textes conservés


,Text,Processed_Text
0,\nCat S60 .. أول هاتف ذكي في العالم مع كاميرا ...,Cat اول هاتف ذكي العالم كاميرا حراريه دبي البو...
1,"\n""فيس بوك"" تتيح لمستخدميها التقاط صور بزاوية ...",فيس بوك تتيح لمستخدميها التقاط صور بزاويه درجه...
2,\nدبي حمدي سعد: نما عدد اشتراكات المستهلكين ال...,دبي حمدي سعد نما عدد اشتراكات المستهلكين الافر...


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from collections import Counter

df = pd.read_csv("preprocessed_dataset.csv")
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

all_words = ' '.join(train_df['Processed_Text']).split()
counter = Counter(all_words)
vocab_list = ['<pad>', '<unk>'] + [w for w, c in counter.items() if c >= 3]
word_to_idx = {w: i for i, w in enumerate(vocab_list)}
pad_idx = word_to_idx['<pad>']
unk_idx = word_to_idx['<unk>']

print(f"Vocabulaire : {len(vocab_list)} tokens")

class TextDataset(Dataset):
    def __init__(self, df, word_to_idx, max_len=200):
        self.texts = []
        for text in df['Processed_Text']:
            tokens = text.split()[:max_len]
            ids = [word_to_idx.get(t, unk_idx) for t in tokens]
            ids += [pad_idx] * (max_len - len(ids))
            self.texts.append(torch.tensor(ids))
        self.scores = torch.tensor(df['Score'].values, dtype=torch.float)

    def __len__(self): return len(self.texts)
    def __getitem__(self, idx): return self.texts[idx], self.scores[idx]

train_dataset = TextDataset(train_df, word_to_idx)
test_dataset = TextDataset(test_df, word_to_idx)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)
print("DataLoaders prêts")

Vocabulaire : 24269 tokens
DataLoaders prêts


In [5]:
import torch.nn as nn

class SeqModel(nn.Module):
    def __init__(self, vocab_size, embed_size=128, hidden_size=256, num_layers=2, model_type='LSTM', bidirectional=False):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=pad_idx)
        direction = 2 if bidirectional else 1
        if model_type == 'RNN':
            self.rnn = nn.RNN(embed_size, hidden_size, num_layers, bidirectional=bidirectional, batch_first=True, dropout=0.3)
        elif model_type == 'GRU':
            self.rnn = nn.GRU(embed_size, hidden_size, num_layers, bidirectional=bidirectional, batch_first=True, dropout=0.3)
        else:
            self.rnn = nn.LSTM(embed_size, hidden_size, num_layers, bidirectional=bidirectional, batch_first=True, dropout=0.3)
        self.fc = nn.Linear(hidden_size * direction, 1)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        out, _ = self.rnn(x)
        return self.fc(out[:, -1, :]).squeeze()

vocab_size = len(vocab_list)

models_dict = {
    'RNN': SeqModel(vocab_size, model_type='RNN'),
    'BiRNN': SeqModel(vocab_size, model_type='RNN', bidirectional=True),
    'GRU': SeqModel(vocab_size, model_type='GRU'),
    'LSTM': SeqModel(vocab_size, model_type='LSTM')
}

def train_model(model, loader, epochs=8, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for texts, scores in loader:
            optimizer.zero_grad()
            outputs = model(texts)
            loss = criterion(outputs, scores)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            total_loss += loss.item()
        if (epoch + 1) % 4 == 0:
            print(f"   Epoch {epoch+1}/{epochs} - Loss: {total_loss / len(loader):.4f}")

for name, model in models_dict.items():
    print(f"\nEntraînement {name}")
    train_model(model, train_loader, epochs=8)
    torch.save(model.state_dict(), f"{name}_model.pth")
print("Entraînement terminé")


Entraînement RNN
   Epoch 4/8 - Loss: 1.3573
   Epoch 8/8 - Loss: 1.3718

Entraînement BiRNN
   Epoch 4/8 - Loss: 1.3937
   Epoch 8/8 - Loss: 1.3087

Entraînement GRU
   Epoch 4/8 - Loss: 1.4362
   Epoch 8/8 - Loss: 1.2405

Entraînement LSTM
   Epoch 4/8 - Loss: 1.3578
   Epoch 8/8 - Loss: 1.2820
Entraînement terminé


In [6]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate(model, loader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for texts, scores in loader:
            outputs = model(texts)
            preds.extend(outputs.cpu().tolist())
            trues.extend(scores.cpu().tolist())
    mse = mean_squared_error(trues, preds)
    mae = mean_absolute_error(trues, preds)
    r2 = r2_score(trues, preds)
    return mse, mae, r2

print("Évaluation :\n")
for name, model in models_dict.items():
    model.load_state_dict(torch.load(f"{name}_model.pth"))
    mse, mae, r2 = evaluate(model, test_loader)
    print(f"{name} → MSE: {mse:.4f} | MAE: {mae:.4f} | R²: {r2:.4f}")

Évaluation :

RNN → MSE: 1.3614 | MAE: 1.0107 | R²: -0.0010
BiRNN → MSE: 1.5317 | MAE: 1.0627 | R²: -0.1263
GRU → MSE: 1.4084 | MAE: 1.0207 | R²: -0.0356
LSTM → MSE: 1.4338 | MAE: 1.0286 | R²: -0.0543


In [7]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

model_name = "aubmindlab/aragpt2-base"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(model_name)

with open("train_arabic.txt", "w", encoding="utf-8") as f:
    for text in df['Text'].head(1500):
        f.write(text.strip() + "\n\n")

dataset = TextDataset(tokenizer=tokenizer, file_path="train_arabic.txt", block_size=128)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./aragpt2_finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    fp16=True,
)

trainer = Trainer(model=model, args=training_args, data_collator=data_collator, train_dataset=dataset)
trainer.train()
trainer.save_model("./aragpt2_finetuned")
tokenizer.save_pretrained("./aragpt2_finetuned")
print("Fine-tuning terminé")

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]

...

 3


wandb: You chose "Don't visualize my results"


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,4.984300


Fine-tuning terminé


In [8]:
from transformers import pipeline

generator = pipeline('text-generation', model='./aragpt2_finetuned', tokenizer=tokenizer)
prompt = "التطورات الحديثة في الذكاء الاصطناعي تشمل"
generated = generator(prompt, max_length=150, temperature=0.8, do_sample=True)[0]['generated_text']
print("Paragraphe généré :\n", generated)

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Paragraphe généré :
 التطورات الحديثة في الذكاء الاصطناعي تشمل مجموعة واسعة من التطبيقات، والتي تهدف إلى تمكين المستخدمين من الاستفادة من تقنيات الذكاء الاصطناعي.
أعلنت هيئة تنظيم الاتصالات أمس عن إطلاق خدمة جديدة تتيح لمستخدميها الحصول على خدمات الجيل الرابع مجاناً عبر الهواتف الذكية العاملة بنظام التشغيل أندرويد، وذلك خلال مشاركتها في أسبوع جيتكس للتقنية 2017.وقال محمد ناصر الغانم الرئيس التنفيذي لهيئة تنظيم الاتصالات إن هذه الخدمة الجديدة تأتي انطلاقاً من حرص الهيئة على مواكبة أحدث التطورات التقنية التي يشهدها قطاع تقنية المعلومات والاتصالات في دولة الإمارات العربية المتحدة، مشيراً إلى أن هذا التوجه يأتي تماشياً مع استراتيجية الهيئة الرامية إلى الارتقاء بمستوى الخدمات الإلكترونية المقدمة للجمهور بما ينسجم مع رؤية حكومة أبوظبي 2021 الهادفة إلى جعل إمارة أبوظبي مركزاً إقليمياً رائداً للخدمات الإلكترونية.وأضاف الغانم: نحن سعداء بأن نكون جزءاً أساسياً مهما من مشاركة الهيئة في معرض جيتكس لهذا العام، حيث نتطلع قدماً للمشاركة في هذا الحدث المهم الذي يقام للمرة الأولى في المنطقة.وتابع الغان

In [9]:
print("""
Synthèse :

J'ai utilisé le dataset SANAD (catégorie Tech) pour collecter des textes arabes sur la technologie.
Préprocessing : normalisation et suppression des diacritiques avec PyArabic, tokenization, stop words.
Implémentation de RNN, BiRNN, GRU et LSTM pour prédire le score de pertinence.
Évaluation avec MSE, MAE et R².
Fine-tuning d'AraGPT2 et génération d'un paragraphe.

Ce lab m'a permis de maîtriser le NLP arabe avec PyTorch et Transformers, en contournant les problèmes d'installation.
""")


Synthèse :

J'ai utilisé le dataset SANAD (catégorie Tech) pour collecter des textes arabes sur la technologie.
Préprocessing : normalisation et suppression des diacritiques avec PyArabic, tokenization, stop words.
Implémentation de RNN, BiRNN, GRU et LSTM pour prédire le score de pertinence.
Évaluation avec MSE, MAE et R².
Fine-tuning d'AraGPT2 et génération d'un paragraphe.

Ce lab m'a permis de maîtriser le NLP arabe avec PyTorch et Transformers, en contournant les problèmes d'installation.

